In [ ]:
from arango import ArangoClient
client = ArangoClient(hosts="http://localhost:8529")
sys_db = client.db("_system")

In [ ]:
db = client.db("profiling")

In [ ]:
coll = db.collection("events")

In [ ]:
hooks = [
    "SequentialVertexFitter::",
    "rave::VertexFactory::",
    "NewV0Fitter::",
    "genfit::GFRaveVertexFactory::",
    "V0FinderModule::",
]

In [ ]:
from collections import defaultdict
from utils import readlines_reversed

def check_line(line, hooks):
    for s in hooks:
        if s in line:
            return True
    return False

def flush(lst, db_coll):
    db_coll.insert_many(lst, silent=True)

def log_extract(filepath, db_coll):
    queue = []
    size = 200
    event = None
    i = 0
    for line in filter(
        lambda l: check_line(l, hooks),
        readlines_reversed(filepath)
    ):
        [_, fname, t_str, *_] = line.split()
        if "event" in fname:
            if event is not None:
                queue.append(event)
                i+=1
            
            if i == size:
                flush(queue, db_coll)
                queue = []
                i = 0
            
            event = defaultdict(list)
        
        event[fname].append(float(t_str))
    
    flush(queue, db_coll)

        
                
            

In [ ]:
log_extract(
    "/home/p/Philip.Obi/b2/results/current/tracking/"
    "v0ValidationGenerateSample.py.log", 
    coll
)